In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
tf.__version__
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
for dirname, _, fname in os.walk('../input'):
    print(dirname, '...........', len(os.listdir(dirname)))

../input ........... 1
../input/mask-datasets-v1 ........... 1
../input/mask-datasets-v1/Mask_Datasets ........... 2
../input/mask-datasets-v1/Mask_Datasets/Train ........... 2
../input/mask-datasets-v1/Mask_Datasets/Train/Mask ........... 350
../input/mask-datasets-v1/Mask_Datasets/Train/No_mask ........... 400
../input/mask-datasets-v1/Mask_Datasets/Validation ........... 2
../input/mask-datasets-v1/Mask_Datasets/Validation/Mask ........... 150
../input/mask-datasets-v1/Mask_Datasets/Validation/No_mask ........... 200


In [6]:
image_size = 150
batch_size = 32

train_dir = '../input/mask-datasets-v1/Mask_Datasets/Train'
valid_dir = '../input/mask-datasets-v1/Mask_Datasets/Validation'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    shear_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='binary'
    #class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='binary'
    #class_mode='categorical'
)

Found 750 images belonging to 2 classes.
Found 350 images belonging to 2 classes.


In [9]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
  ])

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 64)       

In [10]:
history = model.fit(
    train_generator,
    epochs = 10,
    verbose=1,
    validation_data = valid_generator
)

Train for 24 steps, validate for 11 steps
Epoch 1/10
24/24 [==============================] - 37s 2s/step - loss: 0.7352 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
24/24 [==============================] - 32s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
24/24 [==============================] - 31s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
24/24 [==============================] - 31s 1s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
11/24 [============>.................] - ETA: 11s - loss: 0.6931 - accuracy: 0.5000

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')

def plotting():
    epochs = np.arange(1,len(history.history['loss'])+1)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    plt.figure(figsize=(15,3))
    plt.subplot(121)
    plt.plot(epochs, train_acc, 'r', label='train' )
    plt.plot(epochs, val_acc, 'bo', label='valid')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('accuracy')

    plt.subplot(122)
    plt.plot(epochs, train_loss, 'r', label='train')
    plt.plot(epochs, val_loss, 'bo', label='valid')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.show()
    
plotting()

In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labelssoftmax.txt', 'w') as f:
  f.write(labels)

In [ ]:
!ls

In [ ]:
#saved_model_dir = 'model'
#tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('modelsoftmax30epoch.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!ls